## Graph Neural Networks with Pytorch
## Target: Graph Convolutional Networks
- Original Paper: https://arxiv.org/abs/1609.02907
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gcn.py

In [2]:
import sys
import os
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv

sys.path.append('../')
from utils import *
logger = make_logger(name='gcn_logger')

In [4]:
# Load Cora Dataset
dataset = 'Cora'
path = osp.join(os.getcwd(), '..', 'data', dataset)
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [5]:
# Data Check
# 2708 nodes exist, node feature length is 1433
logger.info(f"Node Feature Matrix Info: # Nodes: {data.x.shape[0]}, # Node Features: {data.x.shape[1]}")
logger.info(f"Target Y Info: Y={data.y.size()}")

# Edge Index
# Graph Connectivity in COO format with shape (2, num_edges) = (2, 10556)
# ([[   0,    0,    0,  ..., 2707, 2707, 2707],
#   [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
logger.info(f"Edge Index Shape: {data.edge_index.shape}")
logger.info(f"Edge Weight: {data.edge_attr}")

# So graph is unweighted and undirected
# Custom Dataset can be made with torch_geometric.data.Data

2021-08-08 22:10:00,724 - gcn_logger - Node Feature Matrix Info: # Nodes: 2708, # Node Features: 1433
2021-08-08 22:10:00,725 - gcn_logger - Target Y Info: Y=torch.Size([2708])
2021-08-08 22:10:00,726 - gcn_logger - Edge Index Shape: torch.Size([2, 10556])
2021-08-08 22:10:00,726 - gcn_logger - Edge Weight: None


In [6]:
# components of data can be optained by dictionary format
print(data.keys)

# Other Attributes
# num_nodes, num_edges, contains_isolated_nodes(), contains_self_loops(), is_directed()

# train_mask denotes against which nodes to train (140 nodes)
print(data.train_mask.sum().item())

['x', 'edge_index', 'y', 'train_mask', 'val_mask', 'test_mask']
140


In [7]:
# 이제 Vanilla GCN을 정의한다.
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(
            in_channels=dataset.num_features, out_channels=16, cached=True, normalize=True)
        self.conv2 = GCNConv(
            in_channels=16, out_channels=dataset.num_classes, cached=True, normalize=True)

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

위 코드에서 확인할 부분은 당연히 **GCNConv**이다.  

GCN Convolutional Layer의 기본 형태는 아래와 같다.  

$$ H^{l+1} = D^{-1/2} \hat{A} D^{1/2} H^l W^l $$  

이 때 $l$ 은 layer 번호를 의미하며, $W$는 parameter이다.  
`add_self_loops=True`로 설정하면 (기본값은 True이긴 하다.) $\hat{A} = A + I$ 로 계산된다.  

`in_channels` 와 `out_channels` 인자는 당연히 Input과 Output의 마지막 Shape을 의미한다.  

D=(num_nodes, num_nodes), A=(num_nodes, num_nodes), H=(num_nodes, in_channels), W=(in_channels, out_channels)  
의 형태를 갖고 있기 때문에 Output의 Shape은 (None, num_nodes, out_channels)이다.  

`forward` 할 때는 edge_index와 edge_weight을 입력해 주어야 한다.  

자세한 정보는 [이 곳](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv)에서 확인할 수 있다.  

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mark that you should also transfer data object to GPU
model, data = GCN().to(device), data.to(device)

# Only perform weight-decay on first convolution.
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)

In [9]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(
        input=model()[data.train_mask], target=data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [10]:
best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}'
    logger.info(log.format(epoch, train_acc, best_val_acc, test_acc))

2021-08-08 22:11:01,464 - gcn_logger - Epoch: 001, Train: 0.2500, Val: 0.1840, Test: 0.1770
2021-08-08 22:11:01,468 - gcn_logger - Epoch: 002, Train: 0.3071, Val: 0.1840, Test: 0.1770
2021-08-08 22:11:01,471 - gcn_logger - Epoch: 003, Train: 0.5857, Val: 0.2600, Test: 0.2990
2021-08-08 22:11:01,475 - gcn_logger - Epoch: 004, Train: 0.5857, Val: 0.2860, Test: 0.3150
2021-08-08 22:11:01,478 - gcn_logger - Epoch: 005, Train: 0.5929, Val: 0.2940, Test: 0.3260
2021-08-08 22:11:01,481 - gcn_logger - Epoch: 006, Train: 0.6571, Val: 0.3280, Test: 0.3520
2021-08-08 22:11:01,484 - gcn_logger - Epoch: 007, Train: 0.6571, Val: 0.3460, Test: 0.3600
2021-08-08 22:11:01,488 - gcn_logger - Epoch: 008, Train: 0.6929, Val: 0.3700, Test: 0.3850
2021-08-08 22:11:01,491 - gcn_logger - Epoch: 009, Train: 0.7500, Val: 0.4260, Test: 0.4430
2021-08-08 22:11:01,495 - gcn_logger - Epoch: 010, Train: 0.8214, Val: 0.4800, Test: 0.4860
2021-08-08 22:11:01,498 - gcn_logger - Epoch: 011, Train: 0.8429, Val: 0.5280, T